In [7]:
import mailparser
import re
import pandas as pd
import io
import glob
import numpy as np

In [179]:

class EmlFileParser(object):
    def __init__(self, file):
        super().__init__()
        self.file = file   

    def parse(self):
        df = self.text_to_df()
        df.columns =['lines'] #Name the dataframe column
        df['A'], df['B'] = df['lines'].str.split('\n',1).str #Split lines 
        df = df.drop_duplicates('A', keep='first') #Drop duplicated lines
        df = df.transpose()# Transpose the Data frame 
        df.columns = df.iloc[1] #Name colums with info in row A
        df = df.reindex(df.index.drop(['A','lines'])).reset_index()
        df = df.filter(regex='SITREP|^J\s|^L\s|^N\s')
        df.iloc[:,0] =df.columns[0]
        df = df.rename(columns={ df.columns[0]: "cross_sitrep"})
        df['cross_sitrep'] = df.cross_sitrep.str.replace('=C9', "E").str.extract("(?:MRCC|CROSS|JRCC) ((?:LA*\s)*[A-Z]+\/\d+/(?:N°)*\d+)")
        df = self.colname_homgeneous(df)
        return df
  
    def parse_tahiti(self):
        df = self.text_to_df()
        df.columns =['lines'] #Name the dataframe column
        df['lines'] = df['lines'].str.strip()
        df['A'], df['B'] = df['lines'].str.split('\n',1).str #Split lines 
        df = df.drop_duplicates('A', keep='first') #Drop duplicated lines
        df = df.transpose()# Transpose the Data frame 
        df.columns = df.iloc[1] #Name colums with info in row A
        df = df.reindex(df.index.drop(['A','lines'])).reset_index()
        df = df.filter(regex='BT|JRCC|^J-|^L-|^N-')
        df.iloc[:,0] =df.columns[0]
        df = df.rename(columns={ df.columns[0]: "cross_sitrep"})
        df['cross_sitrep'] = "TAHITI/"+df['BT'].str.extract('(\d+/\d+)')
        df = df.drop(['BT', 'JRCC TAHITI'], axis=1)
        df = self.colname_homgeneous(df)
        return df
        
    def colname_homgeneous(self, df):
        di={'J - INITIAL ACTIONS TAKEN': 'J - PREMIERES MESURES PRISES',
           'J - PREMIÈRES MESURES PRISES':'J - PREMIERES MESURES PRISES',
           'J – PREMIERES MESURES PRISES':'J - PREMIERES MESURES PRISES',
           'J-    PREMIERES MESURES PRISES':'J - PREMIERES MESURES PRISES',
           'L - CHRONOLOGIE':'L - CHRONOLOGIE ET INSTRUCTIONS DE COORDINATION',
           'L - COORDINATING INSTRUCTIONS':'L - CHRONOLOGIE ET INSTRUCTIONS DE COORDINATION',
           'L-    INSTRUCTIONS POUR LA COORDINATION':'L - CHRONOLOGIE ET INSTRUCTIONS DE COORDINATION',
           'L – CHRONOLOGIE ET INSTRUCTIONS DE COORDINATION':'L - CHRONOLOGIE ET INSTRUCTIONS DE COORDINATION',
           'N - ADDITIONAL INFORMATION':'N - RENSEIGNEMENTS COMPLEMENTAIRES',
           'N - RENSEIGNEMENTS COMPLÉMENTAIRES':'N - RENSEIGNEMENTS COMPLEMENTAIRES',
           'N - RENSEIGNEMENTS COMPLMENTAIRES':'N - RENSEIGNEMENTS COMPLEMENTAIRES',
           'N – RENSEIGNEMENTS COMPLEMENTAIRES':'N - RENSEIGNEMENTS COMPLEMENTAIRES',
           'N-    RENSEIGNEMENTS COMPLEMENTAIRES':'N - RENSEIGNEMENTS COMPLEMENTAIRES'}
        return df.rename(columns=lambda x: x.strip()).rename(columns=di)
        
    
    def eml_to_text(self):
        mail = mailparser.parse_from_file(self.file)
        return mail.body

    def text_to_df(self):
        a =  self.eml_to_text().split('\n\n')
        return pd.DataFrame(a)



class EmlFolderParser(object):
    def __init__(self, folder, output):
        super().__init__()
        self.folder = folder
        self.output = output
    
    def parse_folder(self):
        text_df = pd.DataFrame()
        for filename in glob.glob(self.folder+'/*.eml'):
            if ('TAHITI' in filename)== True:            
                out = EmlFileParser(filename).parse_tahiti()
            else:
                out = EmlFileParser(filename).parse()
            text_df = text_df.append(out, ignore_index=True)
        return text_df
        text_df.to_csv(self.output)
    

In [181]:
a=EmlFolderParser('./liste_sitrep', './liste_sitrep/test_all3.csv').parse_folder()b

C:\Users\elsal\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:34: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)
C:\Users\elsal\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:18: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)


In [185]:
a[a['J - PREMIERES MESURES PRISES'].isnull()]

,J - PREMIERES MESURES PRISES,L - CHRONOLOGIE ET INSTRUCTIONS DE COORDINATION,N - RENSEIGNEMENTS COMPLEMENTAIRES,cross_sitrep
24,NaN,LE 26/12/2017\n03:25 – SUITE NOMBREUX APPELS P...,09:47 - OPÉRATION CLOSE\n01 NAVIRE REMORQUÉ\nBT,NaN
40,NaN,0755 - DEBUT DE MISSION BLS DINARD\n0758 - CON...,NEANT,NaN
98,NaN,1928 - SEMI-RIGIDE POMPIERS SUR ZONE - ENGAGEM...,01 NAVIRE DE PLAISANCE A VOILE DESECHOUE,LA GARDE/2017/2529
122,NaN,14:25 – ENGAGEMENT DE LA SNS 255 « MOISE BEGUE...,15:03 - OPÉRATION CLOSE\n01 NAVIRE DIFFICULTÉ ...,NaN
123,NaN,03H50 - « SC OPALE » AU CONTACT DU REQUÉRANT\n...,04:56 - OPÉRATION CLOSE\n01 NAVIRE REMORQUÉ,NaN
124,NaN,23:21 – SNSM « HARAKA » INDISPONIBLE PAR MANQU...,13:50 - OPÉRATION CLOSE\n01 NAVIRE ÉCHOUÉ,NaN
125,NaN,06:20 - CONTACT PROPRIETAIRE NAVIRE « SAMBIOZ...,06:38 - OPÉRATION CLOSE\n01 NAVIRE ÉCHOUÉ,NaN
126,NaN,10:16 – APPEL AU CODIS POUR ENGEGMENT DE LA GA...,10:41 - OPÉRATION CLOSE\n01 NAVIRE DIFFICULTÉ ...,NaN
127,NaN,09:09 - DIFFUSION MESSAGE MAYDAY RELAY VIA VHF...,09:48 - OPÉRATION CLOSE\n01 NAVIRE REMORQUÉ,NaN
135,NaN,0712 – APPELS AU NAVIRE VHF/TEL INFRUCTUEUX\n0...,"01 NAVIRE DE PLAISANCE A VOILE NON ASSISTE, CA...",NaN


In [313]:
def parse_sitrep(input_path, output_path):
    text_df = pd.DataFrame() 
    for filename in glob.glob(input_path+'/*.eml'):
        mail = mailparser.parse_from_file(filename)
        text = mail.body
        a = text.split('\n\n')
        df = pd.DataFrame(a)
        df.columns =['test']
        df['A'], df['B'] = df['test'].str.split('\n',1).str 
        df = df.drop_duplicates('A', keep='first')
        df1 = df.transpose()
        df1.columns = df1.iloc[1]
        df2 = df1.reindex(df1.index.drop('A')).reset_index()
        df3 = df2.filter(regex='SITREP|^J\s|^L\s|^N\s').drop(0)
        df3.iloc[:,0] =df3.columns[0]
        df3 = df3.rename(columns={ df3.columns[0]: "cross_sitrep"})
        df3['cross_sitrep'] = df3.cross_sitrep.str.extract("(?:MRCC|CROSS|JRCC) ([A-Z]+\/\d+/(?:N°)*\d+)")#ajouter un truc pour La garde 
        text_df = text_df.append(df3, ignore_index=True)
    text_df.to_csv(output_path)


    

In [357]:
parse_sitrep('./liste_sitrep', './liste_sitrep/test_all2.csv')

/Users/elsa.luneville/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:17: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)


In [165]:
class EmlFileParserTest(object):
    def __init__(self, file):
        super().__init__()
        self.file = file   

    
    def parse(self):
        df = self.text_to_df()
        df.columns =['lines'] #Name the dataframe column
        df['A'], df['B'] = df['lines'].str.split('\n',1).str #Split lines 
        df = df.drop_duplicates('A', keep='first') #Drop duplicated lines
        df = df.transpose()# Transpose the Data frame 
        df.columns = df.iloc[1] #Name colums with info in row A
        df = df.reindex(df.index.drop(['A','lines'])).reset_index()
        df = df.filter(regex='SITREP|^J\s|^L\s|^N\s')
        df.iloc[:,0] =df.columns[0]
        df = df.rename(columns={ df.columns[0]: "cross_sitrep"})
        df['cross_sitrep'] = df.cross_sitrep.str.replace('=C9', "E").str.extract("(?:MRCC|CROSS|JRCC) ((?:LA*\s)*[A-Z]+\/\d+/(?:N°)*\d+)")
        df = self.colname_homgeneous(df)
        return df
  
    def parse_tahiti(self):
        df = self.text_to_df()
        df.columns =['lines'] #Name the dataframe column
        df['lines'] = df['lines'].str.strip()
        df['A'], df['B'] = df['lines'].str.split('\n',1).str #Split lines 
        df = df.drop_duplicates('A', keep='first') #Drop duplicated lines
        df = df.transpose()# Transpose the Data frame 
        df.columns = df.iloc[1] #Name colums with info in row A
        df = df.reindex(df.index.drop(['A','lines'])).reset_index()
        df = df.filter(regex='BT|JRCC|^J-|^L-|^N-')
        df.iloc[:,0] =df.columns[0]
        df = df.rename(columns={ df.columns[0]: "cross_sitrep"})
        df['cross_sitrep'] = "TAHITI/"+df['BT'].str.extract('(\d+/\d+)')
        df = df.drop(['BT', 'JRCC TAHITI'], axis=1)
        df = self.colname_homgeneous(df)
        return df
        
    def colname_homgeneous(self, df):
        di={'J - INITIAL ACTIONS TAKEN': 'J - PREMIERES MESURES PRISES',
           'J - PREMIÈRES MESURES PRISES':'J - PREMIERES MESURES PRISES',
           'J – PREMIERES MESURES PRISES':'J - PREMIERES MESURES PRISES',
           'J- PREMIERES MESURES PRISES':'J - PREMIERES MESURES PRISES',
           'L - CHRONOLOGIE':'L - CHRONOLOGIE ET INSTRUCTIONS DE COORDINATION',
           'L - COORDINATING INSTRUCTIONS':'L - CHRONOLOGIE ET INSTRUCTIONS DE COORDINATION',
           'L- INSTRUCTIONS POUR LA COORDINATION':'L - CHRONOLOGIE ET INSTRUCTIONS DE COORDINATION',
           'L – CHRONOLOGIE ET INSTRUCTIONS DE COORDINATION':'L - CHRONOLOGIE ET INSTRUCTIONS DE COORDINATION',
           'N - ADDITIONAL INFORMATION':'N - RENSEIGNEMENTS COMPLEMENTAIRES',
           'N - RENSEIGNEMENTS COMPLÉMENTAIRES':'N - RENSEIGNEMENTS COMPLEMENTAIRES',
           'N - RENSEIGNEMENTS COMPLMENTAIRES':'N - RENSEIGNEMENTS COMPLEMENTAIRES',
           'N – RENSEIGNEMENTS COMPLEMENTAIRES':'N - RENSEIGNEMENTS COMPLEMENTAIRES',
           'N- RENSEIGNEMENTS COMPLEMENTAIRES':'N - RENSEIGNEMENTS COMPLEMENTAIRES'}
        return df.rename(columns=di)
        
    
    def eml_to_text(self):
        mail = mailparser.parse_from_file(self.file)
        return mail.body

    def text_to_df(self):
        a =  self.eml_to_text().split('\n\n')
        return pd.DataFrame(a)



class EmlFolderParser(object):
    def __init__(self, folder, output):
        super().__init__()
        self.folder = folder
        self.output = output
    
    def parse_folder(self):
        text_df = pd.DataFrame()
        for filename in glob.glob(self.folder+'/*.eml'):
            if ('TAHITI' in filename)== True:            
                out = EmlFileParser(filename).parse_tahiti()
            else:
                out = EmlFileParser(filename).parse()
            text_df = text_df.append(out, ignore_index=True)
        return text_df
        text_df.to_csv(self.output)

In [166]:
EmlFileParserTest('./liste_sitrep/SITREP DIV UN ET FINAL - CROSS R╔UNION-2017-0836.eml').parse()

C:\Users\elsal\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:18: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)


A,cross_sitrep,L - CHRONOLOGIE ET INSTRUCTIONS DE COORDINATION,N - RENSEIGNEMENTS COMPLEMENTAIRES
0,NaN,LE 26/12/2017\n03:25 – SUITE NOMBREUX APPELS P...,09:47 - OPÉRATION CLOSE\n01 NAVIRE REMORQUÉ\nBT


In [161]:
#Handle cross reunion
#Add message error
#Split parsing function
#make test

C:\Users\elsal\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:18: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)


A,cross_sitrep,L - CHRONOLOGIE ET INSTRUCTIONS DE COORDINATION,N - RENSEIGNEMENTS COMPLEMENTAIRES
0,NaN,LE 26/12/2017\n03:25 – SUITE NOMBREUX APPELS P...,09:47 - OPÉRATION CLOSE\n01 NAVIRE REMORQUÉ\nBT
